In [2]:
# import chromadb
# import ollama
# import pandas as pd
# import os
# import re

# # ==========================
# # 🔹 0. Resume Parser
# # ==========================
# import pdfplumber
# import docx2txt

# def parse_resume(file_path: str) -> str:
#     ext = os.path.splitext(file_path)[1].lower()
#     text = ""
#     if ext == ".pdf":
#         with pdfplumber.open(file_path) as pdf:
#             for page in pdf.pages:
#                 text += page.extract_text() + "\n"
#     elif ext in [".doc", ".docx"]:
#         text = docx2txt.process(file_path)
#     else:
#         raise ValueError("Unsupported file format. Please use PDF, DOC, or DOCX.")
#     # Clean text
#     text = re.sub(r"\s+", " ", text).strip()
#     return text

# # ==========================
# # 🔹 1. Load Internship CSV
# # ==========================
# internships_df = pd.read_csv("internship_grok.csv")  # your dataset

# # Create a combined description for embedding
# internships_df['description'] = (
#     internships_df['internship_title'].astype(str) + " at " +
#     internships_df['company_name'].astype(str) + " | Location: " +
#     internships_df['location'].astype(str) + " | Duration: " +
#     internships_df['duration'].astype(str) + " | Stipend: " +
#     internships_df['stipend'].astype(str)
# )

# # ==========================
# # 🔹 2. Candidate Resume Upload
# # ==========================
# resume_path = "resume.pdf"  # 📂 put your resume file here
# candidate_resume = parse_resume(resume_path)
# print("📄 Parsed Resume (first 300 chars):", candidate_resume[:300], "...\n")

# # ==========================
# # 🔹 3. Embedding Function (Local via Ollama)
# # ==========================
# class OllamaEmbeddingFunction:
#     def __init__(self, model: str = "mxbai-embed-large"):
#         self.model = model

#     def __call__(self, input):
#         if isinstance(input, str):
#             input = [input]
#         embeddings = []
#         for t in input:
#             response = ollama.embeddings(model=self.model, prompt=t)
#             embeddings.append(response["embedding"])
#         return embeddings

# embedding_fn = OllamaEmbeddingFunction(model="mxbai-embed-large")

# # ==========================
# # 🔹 4. Persistent ChromaDB Setup
# # ==========================
# PERSIST_DIR = "chroma_storage"
# chroma_client = chromadb.PersistentClient(path=PERSIST_DIR)

# # Use existing collection if already created
# try:
#     collection = chroma_client.get_collection(
#         name="internships", embedding_function=embedding_fn
#     )
#     print("📂 Loaded existing Chroma collection from disk")
# except:
#     collection = chroma_client.create_collection(
#         name="internships", embedding_function=embedding_fn
#     )
#     print("🆕 Created new Chroma collection")

# # ==========================
# # 🔹 5. Chunking Function
# # ==========================
# def chunk_text(text, chunk_size=300, overlap=50):
#     chunks = []
#     start = 0
#     while start < len(text):
#         end = start + chunk_size
#         chunk = text[start:end]
#         chunks.append(chunk.strip())
#         start += chunk_size - overlap
#     return chunks

# # ==========================
# # 🔹 6. Chunk + Add Internship Descriptions (only if empty)
# # ==========================
# if collection.count() == 0:
#     BATCH_SIZE = 500
#     all_docs, all_ids = [], []

#     for idx, desc in enumerate(internships_df['description'].tolist()):
#         chunks = chunk_text(desc, chunk_size=300, overlap=50)
#         for j, chunk in enumerate(chunks):
#             all_docs.append(chunk)
#             all_ids.append(f"{idx}_{j}")  # unique ID: internshipIndex_chunkIndex

#     for i in range(0, len(all_docs), BATCH_SIZE):
#         batch_docs = all_docs[i:i+BATCH_SIZE]
#         batch_ids = all_ids[i:i+BATCH_SIZE]
#         collection.add(documents=batch_docs, ids=batch_ids)
#         print(f"✅ Inserted batch {i//BATCH_SIZE + 1} with {len(batch_docs)} chunks")
# else:
#     print(f"⚡ Skipping insert: {collection.count()} chunks already stored")

# # ==========================
# # 🔹 7. Chunk Candidate Resume
# # ==========================
# resume_chunks = chunk_text(candidate_resume, chunk_size=400, overlap=50)
# print(f"📑 Resume split into {len(resume_chunks)} chunks")

# # ==========================
# # 🔹 8. Query Matches for Resume Chunks
# # ==========================
# top_n = 5
# all_matches = []

# for chunk in resume_chunks:
#     results = collection.query(query_texts=[chunk], n_results=top_n)
#     top_ids = results['ids'][0]
#     top_texts = results['documents'][0]
#     top_scores = results.get('distances', [[]])[0]

#     raw_confidences = [1 - s for s in top_scores]
#     min_c, max_c = min(raw_confidences), max(raw_confidences)
#     normalized_confidences = [
#         100 * (c - min_c) / (max_c - min_c + 1e-8) for c in raw_confidences
#     ]

#     for iid, text, conf in zip(top_ids, top_texts, normalized_confidences):
#         all_matches.append((iid, text, conf))

# # Deduplicate + sort by confidence
# all_matches = sorted(all_matches, key=lambda x: x[2], reverse=True)
# seen = set()
# unique_matches = []
# for iid, text, conf in all_matches:
#     if iid not in seen:
#         unique_matches.append((iid, text, conf))
#         seen.add(iid)
#     if len(unique_matches) >= top_n:
#         break

# print("\n🔎 Top Internship Matches (with Confidence %):")
# for idx, (iid, text, conf) in enumerate(unique_matches, 1):
#     print(f"{idx}. ID {iid} | Confidence: {conf:.1f}%")
#     print(f"   {text[:120]}...\n")

# # ==========================
# # 🔹 9. RAG Explanation using LLaMA3.1
# # ==========================
# context = "\n\n".join([m[1] for m in unique_matches])
# prompt = f"""
# You are a resume-job matcher.
# Context (Top {top_n} internship chunks):
# {context}

# Task:
# Match the candidate's resume to the above internships and explain why the candidate is suitable.
# Resume:
# {candidate_resume}
# """

# response = ollama.chat(
#     model="llama3.1:8b",
#     messages=[{"role": "user", "content": prompt}],
# )

# print("\n✅ RAG Explanation:\n")
# print(response['message']['content'])

In [3]:
# ==========================
# 🔹 Imports
# ==========================
import pandas as pd
import os
import re
import pdfplumber
import docx2txt
import chromadb
import ollama

In [4]:
# ==========================
# 🔹 0. Resume Parser
# ==========================
def parse_resume(file_path: str) -> str:
    ext = os.path.splitext(file_path)[1].lower()
    text = ""
    if ext == ".pdf":
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    elif ext in [".doc", ".docx"]:
        text = docx2txt.process(file_path)
    else:
        raise ValueError("Unsupported file format. Please use PDF, DOC, or DOCX.")
    # Clean text
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [5]:
# ==========================
# 🔹 1. Load Internship CSV (Modified for your dataset)
# ==========================
internships_df = pd.read_csv("internships_generated.csv")  # 🔹 MODIFIED

# Create a combined description for embedding (focus on relevant features: skills, location, qualification)
internships_df['description'] = (
    internships_df['Internship Title'].astype(str) + " at " +
    internships_df['Company Name'].astype(str) + " | Location: " +
    internships_df['Location'].astype(str) + ", " +
    internships_df['State/UT'].astype(str) + " | Skills: " +
    internships_df['Preferred Skill(s)'].astype(str) + " | Qualification: " +
    internships_df['Minimum Qualification'].astype(str)
)

# ==========================
# 🔹 2. Candidate Resume Upload
# ==========================
resume_path = "resume1.docx"  # 📂 put your resume file here
candidate_resume = parse_resume(resume_path)
print("📄 Parsed Resume (first 300 chars):", candidate_resume[:300], "...\n")

📄 Parsed Resume (first 300 chars): Pooja Patnaik Address: Green Hills Road, Moosapet, Hyderabad Phone: 9110574275 E-Mail: patnaik.pooja9@gmail.com LinkedIn: www.linkedin.com/in/pooja-patnaik-74b367331 Profile As a final-year B. Tech student in CSE (AI & ML), I bring technical curiosity, disciplined learning, and hands-on project expe ...



In [6]:
print(internships_df.shape)
print(internships_df.columns)
print(internships_df.head())

(200, 21)
Index(['Company Name', 'Internship Title', 'Sector', 'Area/Field',
       'No. of Opportunities', 'Location', 'State/UT', 'District', 'Village',
       'ZIP/Postal Code', 'Minimum Qualification', 'Course', 'Specialization',
       'Certification(s)', 'Preferred Skill(s)', 'Qualification Description',
       'Benefits', 'Benefits Description', 'Description',
       'Candidates Already Applied', 'description'],
      dtype='object')
         Company Name               Internship Title          Sector  \
0  Adani Green Energy             Social Work Intern  NGO/Non-Profit   
1       Bharti Airtel            Data Analyst Intern     Hospitality   
2              Swiggy  Hospitality Operations Intern   Manufacturing   
3     Microsoft India          Lab Technician Intern     Hospitality   
4       Maruti Suzuki          Graphic Design Intern      Technology   

                        Area/Field  No. of Opportunities Location  \
0                Digital Marketing                   

In [7]:
# ==========================
# 🔹 3. Embedding Function (Local via Ollama)
# ==========================
class OllamaEmbeddingFunction:
    def __init__(self, model: str = "mxbai-embed-large"):
        self.model = model

    def __call__(self, input):
        if isinstance(input, str):
            input = [input]
        embeddings = []
        for t in input:
            response = ollama.embeddings(model=self.model, prompt=t)
            embeddings.append(response["embedding"])
        return embeddings

embedding_fn = OllamaEmbeddingFunction(model="mxbai-embed-large")

# ==========================
# 🔹 4. Persistent ChromaDB Setup
# ==========================
PERSIST_DIR = "chroma_storage"
chroma_client = chromadb.PersistentClient(path=PERSIST_DIR)

# Use existing collection if already created
try:
    collection = chroma_client.get_collection(
        name="internships", embedding_function=embedding_fn
    )
    print("📂 Loaded existing Chroma collection from disk")
except:
    collection = chroma_client.create_collection(
        name="internships", embedding_function=embedding_fn
    )
    print("🆕 Created new Chroma collection")

# ==========================
# 🔹 5. Chunking Function
# ==========================
def chunk_text(text, chunk_size=300, overlap=50):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk.strip())
        start += chunk_size - overlap
    return chunks

# ==========================
# 🔹 6. Chunk + Add Internship Descriptions
# ==========================
if collection.count() == 0:
    BATCH_SIZE = 500
    all_docs, all_ids = [], []

    for idx, desc in enumerate(internships_df['description'].tolist()):
        chunks = chunk_text(desc, chunk_size=300, overlap=50)
        for j, chunk in enumerate(chunks):
            all_docs.append(chunk)
            all_ids.append(f"{idx}_{j}")

    for i in range(0, len(all_docs), BATCH_SIZE):
        batch_docs = all_docs[i:i+BATCH_SIZE]
        batch_ids = all_ids[i:i+BATCH_SIZE]
        collection.add(documents=batch_docs, ids=batch_ids)
        print(f"✅ Inserted batch {i//BATCH_SIZE + 1} with {len(batch_docs)} chunks")
else:
    print(f"⚡ Skipping insert: {collection.count()} chunks already stored")

# ==========================
# 🔹 7. Chunk Candidate Resume
# ==========================
resume_chunks = chunk_text(candidate_resume, chunk_size=400, overlap=50)
print(f"📑 Resume split into {len(resume_chunks)} chunks")

🆕 Created new Chroma collection
✅ Inserted batch 1 with 200 chunks
📑 Resume split into 15 chunks


In [8]:
# ==========================
# 🔹 8. Query Matches for Resume Chunks (Skills & Location Focus)
# ==========================
top_n = 5
all_matches = []

for chunk in resume_chunks:
    results = collection.query(query_texts=[chunk], n_results=top_n)
    top_ids = results['ids'][0]
    top_texts = results['documents'][0]
    top_scores = results.get('distances', [[]])[0]

    raw_confidences = [1 - s for s in top_scores]
    min_c, max_c = min(raw_confidences), max(raw_confidences)
    normalized_confidences = [
        100 * (c - min_c) / (max_c - min_c + 1e-8) for c in raw_confidences
    ]

    for iid, text, conf in zip(top_ids, top_texts, normalized_confidences):
        # 🔹 Filter based on skills and location match (reduce bias)
        if any(skill.lower() in candidate_resume.lower() for skill in text.split("Skills:")[-1].split("|")[0].split(",")) \
           or any(loc.lower() in candidate_resume.lower() for loc in text.split("Location:")[-1].split("|")[0].split(",")):
            all_matches.append((iid, text, conf))

# Deduplicate + sort by confidence
all_matches = sorted(all_matches, key=lambda x: x[2], reverse=True)
seen = set()
unique_matches = []
for iid, text, conf in all_matches:
    if iid not in seen:
        unique_matches.append((iid, text, conf))
        seen.add(iid)
    if len(unique_matches) >= top_n:
        break

print("\n🔎 Top Internship Matches (with Confidence %):")
for idx, (iid, text, conf) in enumerate(unique_matches, 1):
    print(f"{idx}. ID {iid} | Confidence: {conf:.1f}%")
    print(f"   {text[:120]}...\n")

# ==========================
# 🔹 9. RAG Explanation using LLaMA3.1
# ==========================
context = "\n\n".join([m[1] for m in unique_matches])
prompt = f"""
You are a resume-job matcher.
Context (Top {top_n} internship chunks):
{context}

Task:
Match the candidate's resume to the above internships and explain why the candidate is suitable.
Resume:
{candidate_resume}
"""

response = ollama.chat(
    model="llama3.1:8b",
    messages=[{"role": "user", "content": prompt}],
)

print("\n✅ RAG Explanation:\n")
print(response['message']['content'])


🔎 Top Internship Matches (with Confidence %):
1. ID 114_0 | Confidence: 100.0%
   Data Analyst Intern at Swiggy | Location: Dausa, Rajasthan | Skills: AutoCAD, design | Qualification: Undergraduate...

2. ID 12_0 | Confidence: 100.0%
   Software Development Intern at Myntra | Location: Dehradun, Uttarakhand | Skills: Excel, problem-solving | Qualification...

3. ID 58_0 | Confidence: 100.0%
   Content Writing Intern at NTPC | Location: Mysore, Karnataka | Skills: AutoCAD, design | Qualification: Diploma...

4. ID 26_0 | Confidence: 90.5%
   Software Development Intern at Mindtree | Location: Indore, Madhya Pradesh | Skills: Excel, problem-solving | Qualificat...

5. ID 67_0 | Confidence: 56.5%
   Finance Analyst Intern at Swiggy | Location: Mysore, Karnataka | Skills: Excel, problem-solving | Qualification: Diploma...


✅ RAG Explanation:

Based on the provided internships and Pooja Patnaik's resume, here are my matching suggestions:

**Match 1:** Data Analyst Intern at Swiggy | Locat